<a href="https://colab.research.google.com/github/Vi-vek9135/Task_3_Research_Enhanced/blob/main/Enhanced_Facts_Generation/site/en/tutorials/python_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini API: Quickstart with Python

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/tutorials/python_quickstart"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Google AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This quickstart demonstrates how to use the Python SDK for the Gemini API, which gives you access to Google's Gemini large language models. In this quickstart, you will learn how to:

1. Set up your development environment and API access to use Gemini.
2. Generate text responses from text inputs.
3. Generate text responses from multimodal inputs (text and images).
4. Use Gemini for multi-turn conversations (chat).
5. Use embeddings for large language models.

## Prerequisites

You can run this quickstart in [Google Colab](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb), which runs this notebook directly in the browser and does not require additional environment configuration.

Alternatively, to complete this quickstart locally, ensure that your development environment meets the following requirements:

-  Python 3.9+
-  An installation of `jupyter` to run the notebook.


## Setup


### Install the Python SDK

The Python SDK for the Gemini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:


In [1]:
!pip install -q -U google-generativeai

### Import packages

Import the necessary packages.

In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
# Used to securely store your API key
from google.colab import userdata

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>


In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`


In [4]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## List models

Now you're ready to call the Gemini API. Use `list_models` to see the available Gemini models:

* `gemini-pro`: optimized for text-only prompts.
* `gemini-pro-vision`: optimized for text-and-images prompts.

In [5]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


Note: For detailed information about the available models, including their capabilities and rate limits, see [Gemini models](https://ai.google.dev/models/gemini). There are options for requesting [rate limit increases](https://ai.google.dev/docs/increase_quota). The rate limit for Gemini-Pro models is 60 requests per minute (RPM).

The `genai` package also supports the PaLM  family of models, but only the Gemini models support the generic, multimodal capabilities of the `generateContent` method.

## Generate text from text inputs

For text-only prompts, use the `gemini-pro` model:

In [6]:
model = genai.GenerativeModel('gemini-pro')

The `generate_content` method can handle a wide variety of use cases, including multi-turn chat and multimodal input, depending on what the underlying model supports. The available models only support text and images as input, and text as output.

In the simplest case, you can pass a prompt string to the `GenerativeModel.generate_content` method:

In [7]:
def to_get_response(fact):
    # Assuming model and generate_content function are defined somewhere else in your code
    response = model.generate_content(f"You have to generate Enhanced facts based on fact, for example Fact: Loan pending with the bank, then the Enhanced Fact: A loan is pending with a bank,  then for Query: '{fact}' ", safety_settings={'HARASSMENT':'block_none'})

    prompt = textwrap.dedent(f'''
        Fact: '{fact}'
        Enhanced Fact: '{response.text}'
    ''')

    return prompt

In [8]:
fact = "The accused did not create or submit forged documents to the authority"

In [9]:
prompt = to_get_response(fact)
print(prompt)


Fact: 'The accused did not create or submit forged documents to the authority'
Enhanced Fact: 'Enhanced Fact: The accused is alleged to not have created or submitted forged documents to the authority'



In [10]:
fact = "Loan pending with the bank."

In [11]:
prompt = to_get_response(fact)
print(prompt)


Fact: 'Loan pending with the bank.'
Enhanced Fact: 'Enhanced Fact: A loan is pending with a bank.'



In [12]:
fact = "Physical or mental torture"

In [13]:
prompt = to_get_response(fact)
print(prompt)


Fact: 'Physical or mental torture'
Enhanced Fact: 'Enhanced Fact: Physical or mental torture may be a form of mistreatment and may be a violation of human rights.'



In [11]:
%%time
response = model.generate_content("You have to generate Enhanced facts based on fact, for example Fact: Loan pending with the bank, then the Enhanced Fact: A loan is pending with a bank,  then for Fact: The accused did not create or submit forged documents to the authority ", safety_settings={'HARASSMENT':'block_none'})

CPU times: user 18.4 ms, sys: 3.64 ms, total: 22.1 ms
Wall time: 2.15 s


In [12]:
print(response.text)

Enhanced Fact: The accused did not create or submit fabricated documents to the authority


In simple cases, the `response.text` accessor is all you need. To display formatted Markdown text, use the `to_markdown` function:

In [13]:
to_markdown(response.text)

> Enhanced Fact: The accused did not create or submit fabricated documents to the authority